In [2]:
#!/bin/bash
!kaggle datasets download jithinanievarghese/drugs-side-effects-and-medical-condition

Dataset URL: https://www.kaggle.com/datasets/jithinanievarghese/drugs-side-effects-and-medical-condition
License(s): CC0-1.0




  0%|          | 0.00/1.17M [00:00<?, ?B/s]
 85%|████████▌ | 1.00M/1.17M [00:01<00:00, 738kB/s]
100%|██████████| 1.17M/1.17M [00:01<00:00, 828kB/s]


In [3]:
import zipfile

zip_file_path = "./drugs-side-effects-and-medical-condition.zip"
extract_to_path = "./"

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to_path)
    
print("Extraction complete")

Extraction complete


In [6]:
# Important libraries
from dotenv import load_dotenv
import os
import pandas as pd
from langchain_groq import ChatGroq

# Langchain 
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.output_parsers import ResponseSchema, StructuredOutputParser

In [7]:
# Initialize the model
llm_model = ChatGroq(model="llama-3.3-70b-versatile", api_key=os.environ.get("GROQ_API_KEY"))

In [16]:
context = pd.read_csv("./drugs_side_effects_drugs_com.csv")
context.head(2)

,drug_name,medical_condition,side_effects,generic_name,drug_classes,brand_names,activity,rx_otc,pregnancy_category,csa,alcohol,related_drugs,medical_condition_description,rating,no_of_reviews,drug_link,medical_condition_url
0,doxycycline,Acne,"(hives, difficult breathing, swelling in your ...",doxycycline,"Miscellaneous antimalarials, Tetracyclines","Acticlate, Adoxa CK, Adoxa Pak, Adoxa TT, Alod...",87%,Rx,D,N,X,amoxicillin: https://www.drugs.com/amoxicillin...,Acne Other names: Acne Vulgaris; Blackheads; B...,6.8,760.0,https://www.drugs.com/doxycycline.html,https://www.drugs.com/condition/acne.html
1,spironolactone,Acne,hives ; difficulty breathing; swelling of your...,spironolactone,"Aldosterone receptor antagonists, Potassium-sp...","Aldactone, CaroSpir",82%,Rx,C,N,X,amlodipine: https://www.drugs.com/amlodipine.h...,Acne Other names: Acne Vulgaris; Blackheads; B...,7.2,449.0,https://www.drugs.com/spironolactone.html,https://www.drugs.com/condition/acne.html


In [12]:
response_schemas = [
    ResponseSchema(name="Question", description="The generated question from the provided context"),
    ResponseSchema(name="Answer", description="The corresponding answer from the provided context")
]

output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions(only_json=True)

In [16]:
TEMPLATE = """ You are a helpful agent. Your task is to generate about 50 different or multiple meaningful questions with their corresponding answers using the following provided "{context}" \n\n
"You MUST obey the following criteria:\n"
"- Only focus on different drugs and their side-effects. \n"
"- No pre-amble.\n"
"- Restrict the question to the context information provided.\n"
"- Do NOT create a question that cannot be answered from the context.\n"
"- Phrase the question so that it does NOT refer to specific context.\n"
"- For instance, do NOT use phrases like 'given the provided context' or 'in this work' in the question or 'according to the text' in the answer becuse if the question is asked elsewher it would not be provided specific context. Replace these terms with specific details.\n"
"- Please do NOT repeat the provided context.\n"
"- Please follow JSON recommended format below.\n"
"- Please ensure that the ouput is a valid JSON object.\n"
"{format_instructions}"""

prompt = ChatPromptTemplate.from_template(template=TEMPLATE)


In [24]:
messages = prompt.format_messages(context=context, format_instructions=format_instructions)
response = llm_model.invoke(messages)
response

AIMessage(content='```json\n{\n  "Question": "What are the possible side effects of a drug used to treat Acne?",\n  "Answer": "Possible side effects include hives, difficulty breathing, swelling of the face, lips, tongue, or throat."\n}\n\n{\n  "Question": "What is a common side effect of a weight loss medication?",\n  "Answer": "A common side effect is difficulty breathing."\n}\n\n{\n  "Question": "What are the potential side effects of a tetracycline antibiotic?",\n  "Answer": "Potential side effects include skin rash, fever, swollen glands, and flu-like symptoms."\n}\n\n{\n  "Question": "Can a medication used to treat Acne cause vision or hearing problems?",\n  "Answer": "Yes, some medications used to treat Acne can cause problems with vision or hearing."\n}\n\n{\n  "Question": "What are the possible side effects of a medication that belongs to the class of Anorexiants?",\n  "Answer": "Possible side effects include hives, difficulty breathing, and swelling of the face, lips, tongue,

In [25]:
response_out = response.content
response_out

'```json\n{\n  "Question": "What are the possible side effects of a drug used to treat Acne?",\n  "Answer": "Possible side effects include hives, difficulty breathing, swelling of the face, lips, tongue, or throat."\n}\n\n{\n  "Question": "What is a common side effect of a weight loss medication?",\n  "Answer": "A common side effect is difficulty breathing."\n}\n\n{\n  "Question": "What are the potential side effects of a tetracycline antibiotic?",\n  "Answer": "Potential side effects include skin rash, fever, swollen glands, and flu-like symptoms."\n}\n\n{\n  "Question": "Can a medication used to treat Acne cause vision or hearing problems?",\n  "Answer": "Yes, some medications used to treat Acne can cause problems with vision or hearing."\n}\n\n{\n  "Question": "What are the possible side effects of a medication that belongs to the class of Anorexiants?",\n  "Answer": "Possible side effects include hives, difficulty breathing, and swelling of the face, lips, tongue, or throat."\n}\n\

In [27]:

import json

# Extract the content from the AIMessage object
response_content = response.content

# Remove the Markdown code block markers (```json and ```)
if response_content.startswith('```json') and response_content.endswith('```'):
    response_content = response_content[7:-3].strip()  # Remove ```json and ```
    
print(response_content)


{
  "Question": "What are the possible side effects of a drug used to treat Acne?",
  "Answer": "Possible side effects include hives, difficulty breathing, swelling of the face, lips, tongue, or throat."
}

{
  "Question": "What is a common side effect of a weight loss medication?",
  "Answer": "A common side effect is difficulty breathing."
}

{
  "Question": "What are the potential side effects of a tetracycline antibiotic?",
  "Answer": "Potential side effects include skin rash, fever, swollen glands, and flu-like symptoms."
}

{
  "Question": "Can a medication used to treat Acne cause vision or hearing problems?",
  "Answer": "Yes, some medications used to treat Acne can cause problems with vision or hearing."
}

{
  "Question": "What are the possible side effects of a medication that belongs to the class of Anorexiants?",
  "Answer": "Possible side effects include hives, difficulty breathing, and swelling of the face, lips, tongue, or throat."
}

{
  "Question": "Can a medication 

In [49]:
import json

# Extract the content from the AIMessage object
response_content = response.content

# Remove the Markdown code block markers (```json and ```)
if response_content.startswith('```json') and response_content.endswith('```'):
    response_content = response_content[7:-3].strip()  # Remove ```json and ```

# Split the content into individual JSON objects
json_objects = response_content.split('\n\n')

# Parse and format each JSON object
formatted_content = []
for obj in json_objects:
    try:
        parsed_obj = json.loads(obj)
        formatted_item = {
            "Question": parsed_obj.get("Question", ""),
            "Answer": parsed_obj.get("Answer", "")
        }
        formatted_content.append(formatted_item)
    except json.JSONDecodeError as e:
        print(f"Error parsing JSON object: {e}")

# Save the formatted JSON data to a file
with open('formatted_response_content.json', 'w') as json_file:
    json.dump(formatted_content, json_file, indent=4)

print("Formatted JSON content saved to formatted_response_content.json")

Formatted JSON content saved to formatted_response_content.json


In [58]:
import json

# Read the JSON file
with open('formatted_response_content.json', 'r') as json_file:
    data = json.load(json_file)

# Extract only the questions
questions = [item['Question'] for item in data if 'Question' in item]

qu_ans_list = []


# Print the extracted questions
for question in questions:
    llm_answer = llm_model.invoke(question).content
    qu_ans_pair = {}
    qu_ans_pair["Question"] = question
    qu_ans_pair["Answer"] = llm_answer
    qu_ans_list.append(qu_ans_pair)
    

In [61]:
# Save the formatted JSON data to a file
with open('llm_formatted_response_content.json', 'w') as json_file:
    json.dump(qu_ans_list, json_file, indent=4)

#### Evaluation

In [64]:
import sacrebleu
from rouge_score import rouge_scorer

In [74]:
# Read the JSON file
with open('ref_formatted_response_content.json', 'r') as json_file:
    reference_data = json.load(json_file)
    
with open('llm_formatted_response_content.json', 'r') as json_file:
    llm_data = json.load(json_file)

# Extract only the questions
questions = [item['Question'] for item in reference_data if 'Question' in item]

# Extract only the answer
ref_answers = [item['Answer'] for item in reference_data if 'Answer' in item]
llm_answers = [item['Answer'] for item in llm_data if 'Answer' in item]

print(ref_answers)
print(llm_answers)

# for question in questions:
    

['Possible side effects include hives, difficulty breathing, swelling of the face, lips, tongue, or throat.', 'A common side effect is difficulty breathing.', 'Potential side effects include skin rash, fever, swollen glands, and flu-like symptoms.', 'Yes, some medications used to treat Acne can cause problems with vision or hearing.', 'Possible side effects include hives, difficulty breathing, and swelling of the face, lips, tongue, or throat.', 'Yes, some medications used to treat Weight Loss can cause serious side effects, including hives and difficulty breathing.', 'A potential side effect is difficulty breathing.', 'Yes, some medications used to treat Acne can cause muscle pain.', 'Possible side effects include serious side effects, such as those that affect the face, lips, tongue, or throat.', 'Yes, some medications used to treat Weight Loss can cause swelling of the face, lips, tongue, or throat.', 'Potential side effects include hives, difficulty breathing, and swelling of the f

In [ ]:
for question, ref_ans, llm_ans in zip(questions, ref_answers, llm_answers):
    blue_score = sacrebleu.corpus_bleu([ref_ans], [[llm_ans]]).score
    data_base = {
        "Question": f"{question}\n",
        "LLM Answer": llm_ans,
        "Reference Answer": ref_ans,
        "Blue Score": blue_score
    }
    print(json.dumps(data_base, indent=4))
    

In [ ]:
for question, ref_ans, llm_ans in zip(questions, ref_answers, llm_answers):
    # Initialize the ROUGE scorer
    scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
    rogue_scores = scorer.score(ref_ans, llm_ans)
    data_base = {
        "Question": f"{question}\n",
        "LLM Answer": llm_ans,
        "Reference Answer": ref_ans,
        "Rouge Score":rogue_scores
    }
    print(json.dumps(data_base, indent=4))

**Yes or No Question**

In [84]:
results = []

for question, ref_ans, llm_ans in zip(questions, ref_answers, llm_answers):
    data_base = {
        "Question": f"{question}\n",
        "LLM Answer": llm_ans,
        "Reference Answer": ref_ans,
    }
    results.append(data_base)
    
with open("Question_LLM_Ref_Ans.json", "w") as f:
    json.dump(results, f, indent=4)

In [93]:
# Read out the content of the json file...
with open("Question_LLM_Ref_Ans.json", "r") as read_file:
    context = json.load(read_file)
    
# Print the content
print(json.dumps(context, indent=4))

[
    {
        "Question": "What are the possible side effects of a drug used to treat Acne?\n",
        "LLM Answer": "Drugs used to treat acne can have various side effects, depending on the type of medication, its strength, and individual tolerance. Here are some possible side effects of common acne treatments:\n\n**Topical treatments:**\n\n1. **Benzoyl peroxide**: dryness, redness, irritation, itching, stinging, and burning.\n2. **Salicylic acid**: dryness, redness, irritation, and peeling.\n3. **Retinoids** (e.g., adapalene, tretinoin): redness, irritation, dryness, itching, and sensitivity to the sun.\n4. **Antibiotics** (e.g., clindamycin, erythromycin): skin irritation, redness, itching, and dryness.\n\n**Oral antibiotics:**\n\n1. **Doxycycline**: nausea, vomiting, diarrhea, stomach upset, and increased sensitivity to the sun.\n2. **Minocycline**: dizziness, nausea, vomiting, and stomach upset.\n3. **Erythromycin**: nausea, vomiting, diarrhea, and stomach upset.\n\n**Hormonal 

In [94]:
response_schemas = [
    ResponseSchema(name="Question", description="The generated question from the provided context"),
    ResponseSchema(name="LLM Answer", description="The generated LLM answer from the provided context"),
    ResponseSchema(name="Reference Answer", description="The reference answer from the provided context"),
    ResponseSchema(name="Yes or No Answer", description="Either YES or NO answer generated from the provided context")
]

output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions(only_json=True)

In [133]:
# TEMPLATE = """You are a helpful agent. Your task is to generate five (5) questions, compare the LLM Answer and Reference Answer, reason on the two, and respond with either a YES or NO answer using the provided "{context}". 

# Please follow these criteria:
# - Only generate either a YES or NO answer.
# - Give equal weight and probability to NO and YES answers.
# - Dont add 'Here are five generated questions with YES or NO answers:\n\n'
# - Follow the JSON format below.
# - Ensure the output is a valid JSON object.

# {format_instructions}"""

# prompt = ChatPromptTemplate.from_template(template=TEMPLATE)



TEMPLATE = """
You are a helpful agent. Your task is to generate five (5) questions, compare the LLM Answer and Reference Answer for each question, and reason thoroughly on the two before responding with either a YES or NO answer using the provided "{context}".

Please follow these criteria:
- Only generate either a YES or NO answer.
- Ensure that both answers are thoroughly compared and must really tally before choosing YES or NO.
- Give equal weight and probability to NO and YES answers.
- Dont add 'Here are the generated questions with their corresponding LLM answers, reference answers, and yes or no answers:\n\n\n1.\n'
- Follow the JSON format below.
- Ensure the output is a valid JSON object.

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template=TEMPLATE)


In [134]:
messages = prompt.format_messages(context=context, format_instructions=format_instructions)
response = llm_model.invoke(messages)

In [135]:
# Extract the content from the AIMessage object
response_content = response.content
response_content

'[\n    {\n        "Question": "What are the possible side effects of a drug used to treat Acne?",\n        "LLM Answer": "Drugs used to treat acne can have various side effects, depending on the type of medication, its strength, and individual tolerance. Here are some possible side effects of common acne treatments: Topical treatments: 1. Benzoyl peroxide: dryness, redness, irritation, itching, stinging, and burning. 2. Salicylic acid: dryness, redness, irritation, and peeling. 3. Retinoids (e.g., adapalene, tretinoin): redness, irritation, dryness, itching, and sensitivity to the sun. 4. Antibiotics (e.g., clindamycin, erythromycin): skin irritation, redness, itching, and dryness. Oral antibiotics: 1. Doxycycline: nausea, vomiting, diarrhea, stomach upset, and increased sensitivity to the sun. 2. Minocycline: dizziness, nausea, vomiting, and stomach upset. 3. Erythromycin: nausea, vomiting, diarrhea, and stomach upset. Hormonal treatments: 1. Birth control pills: mood changes, breast

In [137]:
# Remove the Markdown code block markers (```json and ```)
if response_content.startswith('```json') and response_content.endswith('```'):
    response_content = response_content[7:-3].strip()  # Remove ```json and ```

# Parse the JSON content
parsed_content = json.loads(response_content)

# Print the JSON content with indentation
print(json.dumps(parsed_content, indent=4))


[
    {
        "Question": "What are the possible side effects of a drug used to treat Acne?",
        "LLM Answer": "Drugs used to treat acne can have various side effects, depending on the type of medication, its strength, and individual tolerance. Here are some possible side effects of common acne treatments: Topical treatments: 1. Benzoyl peroxide: dryness, redness, irritation, itching, stinging, and burning. 2. Salicylic acid: dryness, redness, irritation, and peeling. 3. Retinoids (e.g., adapalene, tretinoin): redness, irritation, dryness, itching, and sensitivity to the sun. 4. Antibiotics (e.g., clindamycin, erythromycin): skin irritation, redness, itching, and dryness. Oral antibiotics: 1. Doxycycline: nausea, vomiting, diarrhea, stomach upset, and increased sensitivity to the sun. 2. Minocycline: dizziness, nausea, vomiting, and stomach upset. 3. Erythromycin: nausea, vomiting, diarrhea, and stomach upset. Hormonal treatments: 1. Birth control pills: mood changes, breast ten

**Multiple Choice Question and Answers**

In [157]:
response_schemas = [
    ResponseSchema(name="Question", description="The generated multiple-choice question based on the provided context."),
    ResponseSchema(name="Options", description="The list of options(A., B., C., D.) to choose from for the generated question."),
    ResponseSchema(name="Answer", description="The correct answer from the provided options.")
]

output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions(only_json=True)

In [161]:
TEMPLATE = """
You are a helpful agent. Your task is to generate 50 different meaningful multiple-choice questions with their corresponding options and correct answers using the provided "{context}".

You MUST obey the following criteria:
- Only focus on different drugs and their side effects.
- No preamble.
- Restrict the questions to the context information provided.
- Do NOT create a question that cannot be answered from the context.
- Phrase the questions so that they do NOT refer to specific context.
- For instance, do NOT use phrases like 'given the provided context' or 'in this work' in the question or 'according to the text' in the answer because if the question is asked elsewhere it would not be provided specific context. Replace these terms with specific details.
- Do NOT repeat the provided context.
- Ensure that no two questions are the same.
- Randomize the answer options for each question, and ensure no two options are the same.
- Each option must be unique and not repeated across different questions.
- Follow the JSON format below.
- Ensure the output is a valid JSON object.

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template=TEMPLATE)

In [162]:
messages = prompt.format_messages(context=context, format_instructions=format_instructions)
response = llm_model.invoke(messages)
response_content = response.content
response_content

'```json\n{\n    "Questions": [\n        {\n            "Question": "What is a common side effect of the drug doxycycline?",\n            "Options": {\n                "A": "Hives",\n                "B": "Nausea",\n                "C": "Headache",\n                "D": "Fever"\n            },\n            "Answer": "A"\n        },\n        {\n            "Question": "Which of the following is a potential side effect of the drug spironolactone?",\n            "Options": {\n                "A": "Difficulty breathing",\n                "B": "Swollen glands",\n                "C": "Flu-like symptoms",\n                "D": "Skin rash"\n            },\n            "Answer": "A"\n        },\n        {\n            "Question": "What is a possible side effect of the drug minocycline?",\n            "Options": {\n                "A": "Swelling of the face",\n                "B": "Hives",\n                "C": "Fever",\n            "D": "Muscle pain"\n            },\n            "Answer": "B"\n 

In [ ]:
# Remove the Markdown code block markers (```json and ```)
if response_content.startswith('```json') and response_content.endswith('```'):
    response_content = response_content[7:-3].strip()  # Remove ```json and ```

# Parse the JSON content
parsed_content = json.loads(response_content)

# Print the JSON content with indentation
print(json.dumps(parsed_content, indent=4))

In [164]:
with open("Multiple_Choice_Question_LLM_Ref_Ans.json", "w") as f:
    json.dump(parsed_content, f, indent=4)

**Model 2**

In [166]:
import warnings
warnings.filterwarnings("ignore")   # Ignore all warnings

In [171]:
from langchain_community.chat_models import ChatOllama
ollama_llm = ChatOllama(model="llama3.1:8b-instruct-fp16", temperature=0.0, format='json')

In [172]:
response_schemas = [
    ResponseSchema(name="Question", description="The generated multiple-choice question based on the provided context."),
    ResponseSchema(name="Options", description="The list of options(A., B., C., D.) to choose from for the generated question."),
    ResponseSchema(name="Answer", description="The correct answer from the provided options.")
]

output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions(only_json=True)

In [173]:
TEMPLATE = """
You are a helpful agent. Your task is to generate 50 different meaningful multiple-choice questions with their corresponding options and correct answers using the provided "{context}".

You MUST obey the following criteria:
- Only focus on different drugs and their side effects.
- No preamble.
- Restrict the questions to the context information provided.
- Do NOT create a question that cannot be answered from the context.
- Phrase the questions so that they do NOT refer to specific context.
- For instance, do NOT use phrases like 'given the provided context' or 'in this work' in the question or 'according to the text' in the answer because if the question is asked elsewhere it would not be provided specific context. Replace these terms with specific details.
- Do NOT repeat the provided context.
- Ensure that no two questions are the same.
- Randomize the answer options for each question, and ensure no two options are the same.
- Each option must be unique and not repeated across different questions.
- Follow the JSON format below.
- Ensure the output is a valid JSON object.

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template=TEMPLATE)

In [177]:
messages = prompt.format_messages(context=context, format_instructions=format_instructions)
response = ollama_llm.invoke(messages)
response_content = response.content
response_content

ValueError: Ollama call failed with status code 500. Details: {"error":"model requires more system memory (13.0 GiB) than is available (5.6 GiB)"}

In [ ]:
!pip uninstall torch torchvision transformers -y
!pip cache purge
!pip install torch torchvision transformers


**RAG**

In [ ]:
pip install --upgrade transformers

In [1]:
!pip install -q langchain_groq langchain_huggingface langchain_chroma


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from transformers.integrations import integration_utils


In [3]:
# # Import embedding model
from langchain_huggingface import HuggingFaceEmbeddings
embed_model = HuggingFaceEmbeddings(model_name="mixedbread-ai/mxbai-embed-large-v1")

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

c:\Users\Damilare\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Damilare\.cache\huggingface\hub\models--mixedbread-ai--mxbai-embed-large-v1. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/266 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/114k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

In [4]:
# Create a vector store...
from langchain_chroma import Chroma

vectorstore = Chroma(
    collection_name="medical_dataset_store",
    embedding_function=embed_model,
    persist_directory="./",
)
vectorstore.get().keys()

dict_keys(['ids', 'embeddings', 'documents', 'uris', 'data', 'metadatas', 'included'])

In [9]:
# Because the vector store is empty... Add your context data.
vectorstore.add_texts(context)

['3fb3cb0b-f3ab-4113-9d73-8548bc724a90',
 'ee4ffec7-cd79-40ed-898b-3830fe098a45',
 '84179535-4356-491e-bea5-3b830a3c39a5',
 '48af6677-3417-4bd8-b7b8-13683b0a59da',
 'c4f95001-f9fa-4d1f-811a-73a2b187c688',
 '676a96ce-b155-458b-a455-9dfde49e291c',
 '9e5122c1-2aa7-477b-bc07-cebf93695f32',
 'e97c83d9-1021-4b8b-9ebf-889ba8a1cb2c',
 '4e60ecec-2a4a-47a6-b5da-9b9f1579ff2f',
 '68d33a58-1aa1-4aa4-9656-1537b3a33e6f',
 '76e8e59f-76fe-4b92-b0d5-e1c0518c5762',
 '224bd77c-504c-4d1d-8584-7b72ad6ad5cf',
 'b7abc37f-4e43-4d04-9c0a-8fa8ab478f06',
 '5b2c4da0-82e6-4939-b7c5-8b866c792919',
 'c28d6a1e-1bf1-4c1f-ba4f-deec79165374',
 'db8b1908-d791-4097-b811-a67fa892216b',
 'af7407b0-aa6a-4e97-a6fc-24bbff80f346']

In [10]:
query = "What is a common side effect of the drug doxycycline?"
docs = vectorstore.similarity_search(query)
print(docs[0].page_content)
retriever = vectorstore.as_retriever()

side_effects


In [17]:
from langchain_core.prompts import PromptTemplate
template = ("""
You are a medical expert specializing in pharmacology.
Your task is to use the provided context to answer questions about drug side effects for patients.
Please follow these guidelines:
- Provide accurate and detailed answers based on the context.
- If you don't know the answer, clearly state that you don't know.
- Do not reference the context directly in your response; just provide the answer.
- Ensure your answers are clear, concise, and informative.

Context: {context}
Question: {question}
Answer:
""")
rag_prompt = PromptTemplate.from_template(template)

In [18]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | rag_prompt
    | llm_model
    | StrOutputParser()
)

In [19]:
from IPython.display import display, Markdown

response = rag_chain.invoke("What is a common side effect of the drug doxycycline?")
Markdown(response)

One of the common side effects of doxycycline is gastrointestinal upset, which can include symptoms such as nausea, vomiting, and diarrhea. Additionally, doxycycline can also cause photosensitivity, increasing the risk of sunburn and skin rash. Other potential side effects include headache, dizziness, and yeast infections in women. It's essential to note that the severity and frequency of these side effects can vary depending on the individual and the dosage of the medication.

In [32]:
import gradio as gr

# def rag_memory_stream(text):
#     partial_text = ""
#     for new_text in rag_chain.stream(text):
#         partial_text += new_text
#         yield partial_text


# title = "Real-time AI App with Groq API and LangChain to Answer Drug Side Effect Medical Questions"
# demo = gr.Interface(
#     title=title,
#     fn=rag_memory_stream,
#     inputs="text",
#     outputs="text",
#     allow_flagging="never",
# )

# demo.launch(share=True)

import gradio as gr

# Function to handle chatbot logic with memory
def chatbot_with_memory(message, history):
    """
    Respond to the latest user message and update the history.

    Args:
    - message (str): User's latest message.
    - history (list): List of past (user, bot) message tuples.

    Returns:
    - (str, list): Bot response and updated history.
    """
    # Generate bot's response (replace this with your own logic)
    bot_response = f"Echo: {message}"  # Example response, replace with actual chatbot logic.

    # Append user message and bot response to the history
    history.append((message, bot_response))

    # Return the updated history
    return history

# Gradio ChatInterface
demo = gr.ChatInterface(
    fn=chatbot_with_memory,  # Function to process chat
    title="Chatbot with Memory",
    description="This chatbot remembers the conversation history like ChatGPT.",
    examples=["What are the side effects of aspirin?", "Can you tell me about ibuprofen?"],
)

# Launch the application
demo.launch(share=True)

c:\Users\Damilare\AppData\Local\Programs\Python\Python311\Lib\site-packages\gradio\components\chatbot.py:279: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7873
* Running on public URL: https://cfe5cce02c00aabc7a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Traceback (most recent call last):
  File "c:\Users\Damilare\AppData\Local\Programs\Python\Python311\Lib\site-packages\gradio\queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Damilare\AppData\Local\Programs\Python\Python311\Lib\site-packages\gradio\route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Damilare\AppData\Local\Programs\Python\Python311\Lib\site-packages\gradio\blocks.py", line 2042, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Damilare\AppData\Local\Programs\Python\Python311\Lib\site-packages\gradio\blocks.py", line 1587, in call_function
    prediction = await fn(*processed_input)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Damilare\AppData\Local\Programs\Python\Python311\Li